# Script 3: High-Order Interactions Analysis (EEG)
## Analyze high-order statistical interactions (cumulants of order 2, 3, and 4)
## in EEG binary activity, with focus on activity during and outside of avalanches.
## This notebook assumes the output from Scripts 1 and 2 is available.


### 1. Import Libraries
Load libraries for data manipulation and plotting.

In [11]:
import numpy as np
import itertools
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from scipy.stats import moment, skew, kurtosis

### 1. Define Data Path and Load Binary EEG and Avalanche Windows ---
Ensure this path points to the 'eeg_binary.npy' file generated by the simulation notebook.

In [15]:
data_path = "../Data/m_1.0_spont_0.0001/eeg_binary.npy" # Use consistent spontaneous rate

try:
    eeg = np.load(data_path)
    print(f"EEG data loaded successfully. Shape: {eeg.shape}")
except FileNotFoundError:
    print(f"Error: Data file not found at {data_path}.")
    print("Please ensure the simulation script has been run and the path is correct.")
    exit() # Exit if data is not found, as further analysis won't be possible

n_time, n_channels = eeg.shape

# Re-detect avalanches for consistency, similar to the previous script
global_activity = np.any(eeg, axis=1).astype(int)
transitions = np.diff(np.concatenate([[0], global_activity, [0]]))
starts = np.where(transitions == 1)[0]
ends = np.where(transitions == -1)[0]
avalanches = [(start, end) for start, end in zip(starts, ends)]

# Extract activity within avalanches
# Ensure that in_aval is not empty before concatenating
in_aval_segments = [eeg[start:end] for start, end in avalanches if end <= n_time and (end - start) > 0]
if in_aval_segments:
    in_aval = np.vstack(in_aval_segments)
else:
    in_aval = np.array([]) # Set as empty array if no segments
    print("Warning: No 'in_aval' segments found. Cumulant analysis for this state will be skipped.")

# Extract activity outside avalanches (global silence periods)
mask = np.ones(n_time, dtype=bool)
for start, end in avalanches:
    mask[start:end] = False # Mark time steps inside avalanches as False

out_aval = eeg[mask]
if out_aval.shape[0] == 0:
    out_aval = np.array([]) # Set as empty array if no data
    print("Warning: No 'out_aval' samples found (all time steps were active or within an avalanche). Cumulant analysis for this state will be skipped.")


print(f"Samples inside avalanches: {in_aval.shape[0]}")
print(f"Samples outside avalanches: {out_aval.shape[0]}")

EEG data loaded successfully. Shape: (20000, 64)
Samples inside avalanches: 16749
Samples outside avalanches: 3251


### 2. Define High-Order Cumulant Functions ---
These functions compute the generalized cumulants for multivariate data.
For binary data, these are particularly interesting as they reveal deviations from independence.

In [16]:
def compute_second_order_cumulant(X):
    """
    Computes the second-order cumulant (covariance) for all unique pairs of channels.
    Equivalent to covariance for mean-centered data.
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    # Iterate over unique pairs (i, j) where i < j
    for i in range(n_channels):
        for j in range(i, n_channels): # Include i=j for variance (diagonal)
            if i == j: # Variance (diagonal elements)
                cumulants[(i, j)] = np.var(X[:, i])
            else: # Covariance (off-diagonal elements)
                cumulants[(i, j)] = np.cov(X[:, i], X[:, j])[0, 1]
    return cumulants

def compute_third_order_cumulant(X):
    """
    Computes the third-order cumulant for all unique triplets of channels.
    This captures triplet interactions and is related to skewness.
    For mean-centered random variables x, y, z, it's E[xyz].
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    if n_channels < 3:
        return cumulants # Cannot compute 3rd order cumulant for less than 3 channels

    # Mean-center the data once for efficiency
    X_centered = X - np.mean(X, axis=0)

    for comb in itertools.combinations(range(n_channels), 3):
        i, j, k = comb
        val = np.mean(X_centered[:, i] * X_centered[:, j] * X_centered[:, k])
        cumulants[comb] = val
    return cumulants

def compute_fourth_order_cumulant(X):
    """
    Computes the fourth-order cumulant for all unique quadruplets of channels.
    This captures quadruplet interactions and is related to kurtosis.
    For mean-centered random variables x1, x2, x3, x4, it's E[x1x2x3x4] - E[x1x2]E[x3x4] - E[x1x3]E[x2x4] - E[x1x4]E[x2x3].
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    if n_channels < 4:
        return cumulants # Cannot compute 4th order cumulant for less than 4 channels

    # Mean-center the data once for efficiency
    X_centered = X - np.mean(X, axis=0)

    for comb in itertools.combinations(range(n_channels), 4):
        i, j, k, l = comb
        x1 = X_centered[:, i]
        x2 = X_centered[:, j]
        x3 = X_centered[:, k]
        x4 = X_centered[:, l]

        term1 = np.mean(x1 * x2 * x3 * x4)
        term2 = np.mean(x1 * x2) * np.mean(x3 * x4)
        term3 = np.mean(x1 * x3) * np.mean(x2 * x4)
        term4 = np.mean(x1 * x4) * np.mean(x2 * x3)

        val = term1 - term2 - term3 - term4
        cumulants[comb] = val
    return cumulants

### 3. Compute Cumulants Inside and Outside Avalanches ---

In [17]:
cumulants_in_aval_dict = {}
cumulants_out_aval_dict = {}

# Compute 2nd order cumulants (covariance)
if in_aval.shape[0] > 1: # Need at least 2 samples for variance/covariance
    cumulants_in_aval_dict[2] = compute_second_order_cumulant(in_aval)
else:
    print("Skipping 2nd order cumulants 'in_aval' due to insufficient data.")
if out_aval.shape[0] > 1:
    cumulants_out_aval_dict[2] = compute_second_order_cumulant(out_aval)
else:
    print("Skipping 2nd order cumulants 'out_aval' due to insufficient data.")


# Compute 3rd order cumulants (skewness-related)
if in_aval.shape[0] > 2 and n_channels >= 3: # Need at least 3 samples and 3 channels
    cumulants_in_aval_dict[3] = compute_third_order_cumulant(in_aval)
else:
    print("Skipping 3rd order cumulants 'in_aval' due to insufficient data/channels.")
if out_aval.shape[0] > 2 and n_channels >= 3:
    cumulants_out_aval_dict[3] = compute_third_order_cumulant(out_aval)
else:
    print("Skipping 3rd order cumulants 'out_aval' due to insufficient data/channels.")

# Compute 4th order cumulants (kurtosis-related)
if in_aval.shape[0] > 3 and n_channels >= 4: # Need at least 4 samples and 4 channels
    cumulants_in_aval_dict[4] = compute_fourth_order_cumulant(in_aval)
else:
    print("Skipping 4th order cumulants 'in_aval' due to insufficient data/channels.")
if out_aval.shape[0] > 3 and n_channels >= 4:
    cumulants_out_aval_dict[4] = compute_fourth_order_cumulant(out_aval)
else:
    print("Skipping 4th order cumulants 'out_aval' due to insufficient data/channels.")

### 4. Visualize Cumulant Magnitudes ---

In [18]:
output_dir = os.path.dirname(data_path)
os.makedirs(output_dir, exist_ok=True) # Ensure output directory exists

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False) # No shared Y to allow different scales

orders = [2, 3, 4]
titles = ["2nd Order Cumulants (Covariance)", "3rd Order Cumulants (Skewness-related)", "4th Order Cumulants (Kurtosis-related)"]

for i, order in enumerate(orders):
    ax = axes[i]
    cum_in_vals = np.array(list(cumulants_in_aval_dict.get(order, {}).values()))
    cum_out_vals = np.array(list(cumulants_out_aval_dict.get(order, {}).values()))

    # Plotting histograms of absolute cumulant values
    if len(cum_in_vals) > 0:
        ax.hist(np.abs(cum_in_vals), bins=50, density=True, alpha=0.6, label='Inside Avalanches', color='blue', histtype='stepfilled')
    else:
        print(f"No data for plotting {order}th order cumulants 'in_aval'.")

    if len(cum_out_vals) > 0:
        ax.hist(np.abs(cum_out_vals), bins=50, density=True, alpha=0.6, label='Outside Avalanches', color='red', histtype='stepfilled')
    else:
        print(f"No data for plotting {order}th order cumulants 'out_aval'.")

    ax.set_title(titles[i], fontsize=14)
    ax.set_xlabel(f'Absolute Cumulant Value (Order {order})', fontsize=12)
    if i == 0: # Only set ylabel for the first subplot
        ax.set_ylabel('Probability Density', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_yscale('log') # Use log scale for y-axis to better see distributions

plt.suptitle('Comparison of High-Order Cumulants: Inside vs. Outside Avalanches', fontsize=16, y=1.02)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent title overlap
cumulants_plot_path = os.path.join(output_dir, "cumulants_comparison.png")
plt.savefig(cumulants_plot_path, dpi=300)
plt.close()
print(f"\nCumulants comparison plot saved to: {cumulants_plot_path}")

# --- 5. Qualitative Summary of Results ---
print("\n--- Summary of Cumulant Analysis ---")
for order in orders:
    if cumulants_in_aval_dict.get(order) and cumulants_out_aval_dict.get(order):
        mean_in = np.mean(list(cumulants_in_aval_dict[order].values()))
        mean_out = np.mean(list(cumulants_out_aval_dict[order].values()))
        print(f"\nOrder {order} Cumulants:")
        print(f"  Average value inside avalanches: {mean_in:.6f}")
        print(f"  Average value outside avalanches: {mean_out:.6f}")
        # Note: For binary data, cumulants might be small but non-zero.
        # The key is their *relative* magnitude and distribution.
        if order > 2: # For higher orders, non-zero values indicate non-Gaussianity and HOI
            print(f"  Non-zero values for order {order} suggest presence of high-order interactions and non-Gaussian dynamics.")
            if np.abs(mean_in) > np.abs(mean_out):
                print(f"  Higher absolute cumulant values are observed *inside* avalanches, suggesting stronger high-order interactions during active periods.")
            else:
                print(f"  Higher absolute cumulant values are observed *outside* avalanches, suggesting different underlying dynamics during silent periods.")

    elif cumulants_in_aval_dict.get(order):
        mean_in = np.mean(list(cumulants_in_aval_dict[order].values()))
        print(f"\nOrder {order} Cumulants (only inside avalanches data available):")
        print(f"  Average value inside avalanches: {mean_in:.6f}")
    elif cumulants_out_aval_aval_dict.get(order):
        mean_out = np.mean(list(cumulants_out_aval_dict[order].values()))
        print(f"\nOrder {order} Cumulants (only outside avalanches data available):")
        print(f"  Average value outside avalanches: {mean_out:.6f}")
    else:
        print(f"\nOrder {order} Cumulants: No sufficient data to compute for either state.")


print("\nHigh-order interaction analysis complete.")


Cumulants comparison plot saved to: ../Data/m_1.0_spont_0.0001/cumulants_comparison.png

--- Summary of Cumulant Analysis ---

Order 2 Cumulants:
  Average value inside avalanches: 0.015051
  Average value outside avalanches: 0.000000

Order 3 Cumulants:
  Average value inside avalanches: -0.012481
  Average value outside avalanches: 0.000000
  Non-zero values for order 3 suggest presence of high-order interactions and non-Gaussian dynamics.
  Higher absolute cumulant values are observed *inside* avalanches, suggesting stronger high-order interactions during active periods.

Order 4 Cumulants:
  Average value inside avalanches: 0.010168
  Average value outside avalanches: 0.000000
  Non-zero values for order 4 suggest presence of high-order interactions and non-Gaussian dynamics.
  Higher absolute cumulant values are observed *inside* avalanches, suggesting stronger high-order interactions during active periods.

High-order interaction analysis complete.
